In [ ]:
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
import gradio as gr
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
import random
from IPython.display import Image, display

In [ ]:
# Some useful constants

nouns = ["Cabbages", "Unicorns", "Toasters", "Penguins", "Bananas", "Zombies", "Rainbows", "Eels", "Pickles", "Muffins"]
adjectives = ["outrageous", "smelly", "pedantic", "existential", "moody", "sparkly", "untrustworthy", "sarcastic", "squishy", "haunted"]

In [ ]:
load_dotenv(override=True)

In [ ]:
# Annotated is a type hinting tool that allows you to add metadata to your code.
# This can be used by Langraph to understand the type of the input and output of a function.
# It is also used to add documentation to the function.
def shout(text: Annotated[str, "something to be shouted"]) -> str:
    print(text.upper())
    return text.upper()

shout("Hello, world!")

#### Step 1: Define a State Object
#### Step 2: Start the Graph builder
#### Step 3: Create Node
#### Step 4: Create Edges
#### Step 5: Compile the Graph

In [ ]:
# Step 1: Define a State Object
class State(BaseModel):
    messages: Annotated[list, add_messages]

# Step 2: Start the Graph builder
graph_builder = StateGraph(State)

# Step 3: Create Node
def node_1(old_state: State) -> State:
    reply = f"{random.choice(nouns)} are {random.choice(adjectives)}"
    messages = [{"role": "assistant", "content": reply}]
    new_state = State(messages=messages)
    return new_state

graph_builder.add_node("node_1", node_1)

# Step 4: Create Edges
graph_builder.add_edge(START, "node_1")
graph_builder.add_edge("node_1", END)

# Step 5: Compile the Graph
graph = graph_builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

#### Step 6: Using the Compiled Graph

In [ ]:
def chat(user_input: str, history):
    message = {"role": "user", "content": user_input}
    messages = [message]
    state = State(messages=messages)
    result = graph.invoke(state)
    print(result)
    return result["messages"][-1].content

gr.ChatInterface(chat, type="messages").launch()

#### Learnings  
    Langraph does not have to have an LLM its a framework to create and run a Graph.  
    State -> Current state of the application in our example the message.  
    Node -> Functions the takes a state as an input and outputs a new_state.  
    Edges -> Functions which specifies which node to run next  
        - In our example we setup edges using START and END keywords that Langgraph provides  
    Graph -> Structure which consists of multiple Nodes and edges.  
        - Start the GraphBuilder
        - Create nodes and edges using that GraphBuilder
        - Compile the Graph

    LAST STEP -> After all this is done we can invoke this graph.  

    Notice we did not use any LLM anywhere in the this example becasue we were just learning how Langgraph is setup and works.

#### LLM USE

Now we will use LLM in a langraph node.  
Node function will invoke and LLM to generate some output.  
Imagine the possibilities of this when the Graph has more Nodes and Edges.  

In [ ]:
# Step1 : Create a State Object
class State(BaseModel):
    messages: Annotated[list, add_messages]

In [ ]:
# Step 2: Start the Grpah Builder
graph_builder = StateGraph(State)

In [ ]:
# Step 3: Create a Node
llm = ChatOpenAI(model="gpt-4o-mini")
def chatbot_node(old_state: State) -> State:
    response = llm.invoke(old_state.messages)
    new_state = State(messages=[response])
    return new_state

graph_builder.add_node("chatbot", chatbot_node)

In [ ]:
# Step 4: Add Edges
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

In [ ]:
# Step 5: Compile the Graph
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

#### Use the compiled Graph

In [ ]:
def chat(user_input: str, history):
    initial_state = [{"role": "user", "content": user_input}]
    state = State(messages=initial_state)
    result = graph.invoke(state)
    print(result)
    return result["messages"][-1].content

gr.ChatInterface(chat, type="messages").launch()

#### Learnings  
    Memory is not used in this example so its a chat app but without memory.  
    The interface will show the chats that has happened, but if you ask the LLM the question about any past conversation it wont remember.  
    We will solve this in the next practice session.